# Zadania - metody optymalizacji

### 7.1 AdaGrad

1. Rozszerz poniższą implementację algorytmu SGD o funkcjonalność algorytmu AdaGrad (włączaną parametrem `adaGrad=True`). Zasady działania AdaGrad są podane w materiałach do wykładu.
   <br />**Uwaga**: podczas dzielenia przez pierwastek sumy kwadratów historycznych gradientów warto dodać małą wartość $\epsilon=10^{-7}$ do mianownika, aby uniknąć dzielenia przez 0.
   
1. Stwórz model wieloklasowej regresji logistycznej na zbiorze MNIST (dla wszystkich 10 cyfr) i oblicz jego jakość na zbiorze testowym. Zastosuj poniższe parametry:
 1. Rozmiar wsadu: 50
 1. Liczba epok: 2
 1. Rozmiar kroku $\alpha$: 1.0 (w algorytmie AdaGrad $\alpha$ może być niezmienne)
1. Spróbuj uzyskać wynik podobny lub lepszy samym algorytmem SGD bez opcji AdaGrad, dostrajając parametry $\alpha$. Czy jest to możliwe?

### 7.2 Ensemble

1. Na danych MNIST wytrenuj 10 klasyfikatorów z sensownie dobranymi parametrami dla algorytmu SGD. Zastosuj randomizację zbioru uczącego dla każdego modelu i w każdej epoce.<br />
**Uwaga**: pamiętaj, aby tasować $X$ i $Y$ w tej samej kolejności! 
1. Oblicz jakość każdego z klasyfikatorów na zbiorze testowym.
1. Oblicz jakość predykcji (klas) uzyskanych w wyniku wybranej metody agregacji wyników: głosowania klas lub uśredniania prawdopodobieństw, i porównaj z wcześniej uzyskanymi wynikami. Jak wynik z metody zbiorczej odnosi się do wyników uzyskiwanych przez pojedyncze klasyfikatory?

In [ ]:
import numpy as np

def safeSigmoid(x, eps=0):
    y = 1.0/(1.0 + np.exp(-x))
    if eps > 0:
        y[y < eps] = eps
        y[y > 1 - eps] = 1 - eps
    return y

def h(theta, X, eps=0.0):
    return safeSigmoid(X*theta, eps)

def J(h,theta,X,y):
    m = len(y)
    f = h(theta, X, eps=10**-7)
    return -np.sum(np.multiply(y, np.log(f)) + 
                   np.multiply(1 - y, np.log(1 - f)), axis=0)/m

def dJ(h,theta,X,y):
    return 1.0/len(y)*(X.T*(h(theta,X)-y))

def softmax(X):
    return np.exp(X)/np.sum(np.exp(X))

In [ ]:
def SGD(h, fJ, fdJ, theta, X, y, 
        alpha=0.001, maxEpochs=1, batchSize=100):
    m, n = X.shape
    start, end = 0, batchSize
    
    maxSteps = (m * float(maxEpochs)) / batchSize
    for i in range(int(maxSteps)):
        XBatch, yBatch =  X[start:end,:], y[start:end,:]

        theta = theta - alpha * fdJ(h, theta, XBatch, yBatch)
        
        if start + batchSize < m:
            start += batchSize
        else:
            start = 0
        end = min(start + batchSize, m)
        
    return theta